# Visualise Results
This Notebook downloads experiment data from comet-ml and aggregates or visualises them for the thesis.

In [33]:
from comet_ml.api import API
import comet_ml
import pandas as pd
import numpy as np
import seaborn as sns
import json
import matplotlib.pyplot as plt
# set default plt figsize to (12,6)
plt.rcParams["figure.figsize"] = (12, 6)
pd.set_option('display.colheader_justify', 'center')
# Set default font sizes
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['legend.fontsize'] = 16
plt.rcParams['legend.title_fontsize'] = 18


# Initialize the Comet API with API key
api_key = "GAzEuNsoYVpPXO2ryeTk7C1o7"
api = API(api_key=api_key)

In [34]:
def get_experiments_results_data(workspace, project_name, experiment_ids):
    # Get the list of experiments in the project
    result_data = []
    for k, v in experiment_ids.items():
        experiment = api.get(workspace=workspace,
                             project_name=project_name, experiment=v)
        print(experiment.id)
        print(experiment.name)
        # You can also fetch individual metrics, parameters, etc.
        metrics_valid_mrr = experiment.get_metrics(["valid_mrr"])
        metrics_test_mrr = experiment.get_metrics(["test_mrr"])
        max_valid_mrr_value = float('-inf')
        max_epoch = None

        for record in metrics_valid_mrr:
            metric_value = float(record['metricValue'])
            if metric_value > max_valid_mrr_value:
                max_valid_mrr_value = metric_value
                max_epoch = record['epoch']

        print(f"Max valid_mrr: {max_valid_mrr_value}, Epoch: {max_epoch}")
        for record in metrics_test_mrr:
            if record['epoch'] == max_epoch:
                metric_test_mrr_epoch = float(record['metricValue'])
                break
        print(f"Max test_mrr: {metric_test_mrr_epoch}, Epoch: {max_epoch}")

        result_data.append({"name": k, "key": experiment.id, "max_valid_mrr": max_valid_mrr_value,
                           "test_mrr": metric_test_mrr_epoch, "epoch": max_epoch})

    return result_data


def plot_training_curves(df_mrr, title="MRR Curves", visualize_test_mrr=True):
    plt.figure(figsize=(10, 6))
    if visualize_test_mrr:
        sns.lineplot(df_mrr[df_mrr["metricName"] == "test_mrr"], x="epoch",
                     y="metricValue", label="test_mrr", linestyle="dashed", color="red", alpha=0.7)
    sns.lineplot(df_mrr[df_mrr["metricName"] == "train_mrr"],
                 x="epoch", y="metricValue", label="train_mrr", color="green")
    sns.lineplot(df_mrr[df_mrr["metricName"] == "valid_mrr"],
                 x="epoch", y="metricValue", label="valid_mrr", color="blue")

    plt.ylim(0, 1)
    plt.xlabel('Epochs')
    plt.ylabel('MRR')
    plt.title(title)
    plt.legend()
    plt.savefig(f"{title}.pdf", format='pdf',dpi=5000,bbox_inches='tight')

    plt.show()


def plot_training_curves_loss_auc(df_mrr, title="MRR Curves", y_axis="Metrics", ):
    plt.figure(figsize=(10, 6))
    sns.lineplot(df_mrr[df_mrr["metricName"] == "train_mrr"],
                 x="epoch", y="metricValue", label="Train MRR", color="blue")
    sns.lineplot(df_mrr[df_mrr["metricName"] == "train_rocauc"],
                 x="epoch", y="metricValue", label="AUROC", color="orange")
    sns.lineplot(df_mrr[df_mrr["metricName"] == "loss_epoch"], x="epoch",
                 y="metricValue", label="Negative-Log-\nLikelihood-Loss", color="green")

    plt.ylim(0, 1)
    plt.xlabel('Epochs')
    plt.ylabel(y_axis)
    plt.title(title)
    plt.legend()
    plt.show()
def get_experiments_data(workspace, project_name, experiment_ids, other_losses=False, epoch_loss=False):
    # Get the list of experiments in the project
    metrics_data = []
    for k, v in experiment_ids.items():
        experiment = api.get(workspace=workspace,
                             project_name=project_name, experiment=v)
        print(experiment.id)
        print(experiment.name)
        # You can also fetch individual metrics, parameters, etc.
        train_mrr = pd.DataFrame(experiment.get_metrics("train_mrr"))[
            ['metricName', 'metricValue', 'epoch', 'timestamp']]
        valid_mrr = pd.DataFrame(experiment.get_metrics("valid_mrr"))[
            ['metricName', 'metricValue', 'epoch', 'timestamp']]
        test_mrr = pd.DataFrame(experiment.get_metrics("test_mrr"))[
            ['metricName', 'metricValue', 'epoch', 'timestamp']]
        train_loss = pd.DataFrame(experiment.get_metrics("loss_epoch"))[
            ['metricName', 'metricValue', 'epoch', 'timestamp']]

        if other_losses:
            valid_loss = pd.DataFrame(experiment.get_metrics("valid_loss"))[
                ['metricName', 'metricValue', 'epoch', 'timestamp']]
            test_loss = pd.DataFrame(experiment.get_metrics("test_loss"))[
                ['metricName', 'metricValue', 'epoch', 'timestamp']]
            metrics = pd.concat(
                [train_mrr, valid_mrr, test_mrr, train_loss, valid_loss, test_loss])
        elif epoch_loss:
            epoch_loss = pd.DataFrame(experiment.get_metrics("loss_epoch"))[
                ['metricName', 'metricValue', 'epoch', 'timestamp']]
            auc = pd.DataFrame(experiment.get_metrics("train_rocauc"))[
                ['metricName', 'metricValue', 'epoch', 'timestamp']]
            metrics = pd.concat(
                [train_mrr, valid_mrr, test_mrr, epoch_loss, auc])
        else:
            metrics = pd.concat([train_mrr, valid_mrr, test_mrr, train_loss])
        metrics_data.append(
            {"name": k, "key": experiment.id, "metrics": metrics})
    return metrics_data

## Reference Models
Model data for reference models (gcn and gin, sage) is downloaded and visualised.

In [ ]:
experiment_ids = {'GCN reference':'15813a0135394f5fbf73e376f46b4cf9',
                  'GIN reference':'ea2202bee86a4394adbcbc3c48b47ea9',
                  'SAGE reference':'90b05358d9ec47a481eb81ba192b1fad'}
workspace = "tmandelz-outlook-com's Organisation"
project_name = "transfer-learning-link-prediction"



experiment_data = get_experiments_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)
results_data = get_experiments_results_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)

In [ ]:
experiment_ids_2 = {'GCN reference 2':'23ddab6f18b34bdaa97fe203614553df',
                  'GIN reference 2':'e945a7824ad2499c921a028738067cf0',
                  'SAGE reference 2':'830bf7d7d66446f2bbb34d7d7b80c972'}

experiment_data_2 = get_experiments_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids_2)
results_data_2 = get_experiments_results_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids_2)

In [37]:

# get best 
gcn_df = experiment_data[0]['metrics']
gcn_df2 = experiment_data_2[0]['metrics'][:-1]
gcn_df2.loc[:,"epoch"] += gcn_df["epoch"].max()
gcn_df = pd.concat([gcn_df,gcn_df2]).reset_index(drop=True)

firstepoch_test_mrr_gcn_ref = float(gcn_df[gcn_df['metricName'] == "test_mrr"].iloc[0]['metricValue'])

valid_max_idx = gcn_df.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_gcn_ref = float(gcn_df.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])


gin_df = experiment_data[1]['metrics']
gin_df2 = experiment_data_2[1]['metrics'][:-1]
gin_df2.loc[:,"epoch"] += gin_df["epoch"].max()
gin_df = pd.concat([gin_df,gin_df2]).reset_index(drop=True)

firstepoch_test_mrr_gin_ref = float(gin_df[gin_df['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = gin_df.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_gin_ref = float(gin_df.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])


sage_df = experiment_data[2]['metrics']
sage_df2 = experiment_data_2[2]['metrics'][:-1]
sage_df2.loc[:,"epoch"] += sage_df["epoch"].max()
sage_df = pd.concat([sage_df,sage_df2]).reset_index(drop=True)

firstepoch_test_mrr_sage_ref = float(sage_df[sage_df['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = sage_df.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_sage_ref = float(sage_df.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])

### Plot MRR Curves
Plot MRR Curves together into a single visualisation.

In [ ]:
gcn_df['metricValue'] = gcn_df['metricValue'].astype(float)
gin_df['metricValue'] = gin_df['metricValue'].astype(float)
sage_df['metricValue'] = sage_df['metricValue'].astype(float)

plot_training_curves(gin_df, "Reference GCN - MRR Curves")
plot_training_curves(gin_df, "Reference GIN - MRR Curves")
plot_training_curves(sage_df, "Reference SAGE - MRR Curves")

## Finetuning Models
Model data for Finetuning models is downloaded and visualised.

In [ ]:
experiment_ids = {'GCN finetune':'0b67c8710eaa4785bfc1579a341a21db',
                  'GIN finetune':'f5f2c85520b44bb580881868e07b2d76',
                  'SAGE finetune':'dc44f38332824315a070238c399a08fe'}
workspace = "tmandelz-outlook-com's Organisation"
project_name = "transfer-learning-link-prediction"

experiment_data = get_experiments_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)
results_data = get_experiments_results_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)

### Mean Difference Validation and Train MRR
Calculate the mean difference from validation to train MRR.

In [ ]:
df_fine_val_train_mrr =experiment_data[0]['metrics']
df_fine_val_train_mrr = df_fine_val_train_mrr[df_fine_val_train_mrr["metricName"].isin(["train_mrr","valid_mrr"])]
np.round(np.mean(np.array(df_fine_val_train_mrr.pivot(columns=["metricName"])['metricValue']['valid_mrr'],dtype=float) - np.array(df_fine_val_train_mrr.pivot(columns=["metricName"])['metricValue']['train_mrr'],dtype=float)),4)


### Plot MRR Curves
Plot MRR Curves together into a single visualisation.

In [41]:
gcn_df_fine = experiment_data[0]['metrics']
firstepoch_test_mrr_gcn_fine = float(gcn_df_fine[gcn_df_fine['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = gcn_df_fine.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_gcn_fine = float(gcn_df_fine.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])


gin_df_fine = experiment_data[1]['metrics']
firstepoch_test_mrr_gin_fine = float(gin_df_fine[gin_df_fine['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = gin_df_fine.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_gin_fine = float(gin_df_fine.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])

sage_df_fine = experiment_data[2]['metrics']
firstepoch_test_mrr_sage_fine = float(sage_df_fine[sage_df_fine['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = sage_df_fine.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_sage_fine = float(sage_df_fine.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])

In [ ]:
gcn_df_fine['metricValue'] = gcn_df_fine['metricValue'].astype(float)
gin_df_fine['metricValue'] = gin_df_fine['metricValue'].astype(float)
sage_df_fine['metricValue'] = sage_df_fine['metricValue'].astype(float)

plot_training_curves(gcn_df_fine, "arXiv CS Finetuning GCN - MRR Curves")
plot_training_curves(gin_df_fine, "arXiv CS Finetuning GIN - MRR Curves")
plot_training_curves(sage_df_fine, "arXiv CS Finetuning SAGE - MRR Curves")

## Compute Time Differences GCN Reference / Finetune
Computing time differences between the reference and finetuned model

In [ ]:
gcn_df['timestamp'] = gcn_df['timestamp']/1000
gcn_df['timestamp'] = pd.to_datetime(gcn_df['timestamp'], unit='s')
gcn_df['minute_diff'] = (gcn_df['timestamp'] - gcn_df['timestamp'].iloc[0]).dt.total_seconds() / 60 / 60

gcn_df_fine['timestamp'] = gcn_df_fine['timestamp']/1000
gcn_df_fine['timestamp'] = pd.to_datetime(gcn_df_fine['timestamp'], unit='s')
gcn_df_fine['minute_diff'] = (gcn_df_fine['timestamp'] - gcn_df_fine['timestamp'].iloc[0]).dt.total_seconds() / 60 / 60

gin_df['timestamp'] = gin_df['timestamp']/1000
gin_df['timestamp'] = pd.to_datetime(gin_df['timestamp'], unit='s')
gin_df['minute_diff'] = (gin_df['timestamp'] - gin_df['timestamp'].iloc[0]).dt.total_seconds() / 60 / 60

gin_df_fine['timestamp'] = gin_df_fine['timestamp']/1000
gin_df_fine['timestamp'] = pd.to_datetime(gin_df_fine['timestamp'], unit='s')
gin_df_fine['minute_diff'] = (gin_df_fine['timestamp'] - gin_df_fine['timestamp'].iloc[0]).dt.total_seconds() / 60 / 60

sage_df['timestamp'] = sage_df['timestamp']/1000
sage_df['timestamp'] = pd.to_datetime(sage_df['timestamp'], unit='s')
sage_df['minute_diff'] = (sage_df['timestamp'] - sage_df['timestamp'].iloc[0]).dt.total_seconds() / 60 / 60

sage_df_fine['timestamp'] = sage_df_fine['timestamp']/1000
sage_df_fine['timestamp'] = pd.to_datetime(sage_df_fine['timestamp'], unit='s')
sage_df_fine['minute_diff'] = (sage_df_fine['timestamp'] - sage_df_fine['timestamp'].iloc[0]).dt.total_seconds() / 60 / 60

In [ ]:
sage_df[sage_df['metricName'] == "test_mrr"]

In [ ]:
# get training step where the reference model was the first time over 10 hours (SAGE)
sage_df_test_mrr = sage_df[sage_df['metricName'] == "test_mrr"]
sage_df_test_mrr[sage_df_test_mrr['minute_diff'] <=2.01][-1:]


In [ ]:
# get training step where the finetuned model is as close as possible to the mrr of the above training step (GIN)
sage_df_fine_test_mrr = sage_df_fine[sage_df_fine['metricName'] == "test_mrr"]
sage_df_fine_test_mrr[sage_df_fine_test_mrr['metricValue'] >= 0.833613	] # TODO Change hardcode

### Plotting the training time
Plotting the training time and especially the intersection points for the equivalent metrics.


In [ ]:
np.round(2/0.130118,1)

In [ ]:
sns.lineplot(sage_df[sage_df["metricName"] == "test_mrr"],x="minute_diff",y="metricValue", label="SAGE Reference - Test MRR",color="magenta",alpha=0.7)
sns.lineplot(sage_df_fine_test_mrr[sage_df_fine_test_mrr["metricName"] == "test_mrr"],x="minute_diff",y="metricValue", label="SAGE Finetuning - Test MRR",color="cyan",alpha=0.7)
plt.ylim(0,1)
plt.xlim(0,10)
plt.xlabel('Training Time (hours)')
plt.ylabel('Test MRR')
plt.title("Comparison of Training Hours for Transfer Learning")
plt.axhline(0.833613,alpha=0.5,color="green",linestyle="dashed",label="Test MRR ~ 0.83")
plt.plot(2,0.833613,'ro', markersize=7,color="magenta", markeredgecolor='black', label='Reference - 2 Hour Training')
plt.plot(0.130118,0.836982	,'ro', markersize=7,color="cyan", markeredgecolor='black', label='Finetuning - Intersection Point')
plt.legend()

plt.show()

## Visualize Finetune vs Reference

In [ ]:
sns.lineplot(gcn_df[gcn_df["metricName"] == "test_mrr"],x="epoch",y="metricValue", label="GCN Reference - Test MRR",color="blue",alpha=0.7)
sns.lineplot(gcn_df_fine[gcn_df_fine["metricName"] == "test_mrr"],x="epoch",y="metricValue", label="GCN Finetuning - Test MRR",color="red",alpha=0.7)
plt.ylim(0,1)
plt.xlabel('Epoch')
plt.ylabel('Test MRR')
plt.title("Comparison of Reference vs. Finetune \n Model: GCN")
plt.legend()
plt.savefig("gcn_fine_vs_reference.pdf", format='pdf',dpi=5000,bbox_inches='tight')
plt.show()


In [ ]:
sns.lineplot(gin_df[gin_df["metricName"] == "test_mrr"],x="epoch",y="metricValue", label="GIN Reference - Test MRR",color="blue",alpha=0.7)
sns.lineplot(gin_df_fine[gin_df_fine["metricName"] == "test_mrr"],x="epoch",y="metricValue", label="GIN Finetuning - Test MRR",color="red",alpha=0.7)
plt.ylim(0,1)
plt.xlabel('Epoch')
plt.ylabel('Test MRR')
plt.title("Comparison of Reference vs. Finetune \n Model: GIN")
plt.legend()
plt.savefig("gin_fine_vs_reference.pdf", format='pdf',dpi=5000,bbox_inches='tight')
plt.show()

In [ ]:
sns.lineplot(sage_df[sage_df["metricName"] == "test_mrr"],x="epoch",y="metricValue", label="SAGE Reference - Test MRR",color="blue",alpha=0.7)
sns.lineplot(sage_df_fine[sage_df_fine["metricName"] == "test_mrr"],x="epoch",y="metricValue", label="SAGE Finetuning - Test MRR",color="red",alpha=0.7)
plt.ylim(0,1)
plt.xlabel('Epoch')
plt.ylabel('Test MRR')
plt.title("Comparison of Reference vs. Finetune \n Model: SAGE")
plt.savefig("sage_fine_vs_reference.pdf", format='pdf',dpi=5000,bbox_inches='tight')

plt.legend()
plt.show()

## Standard Deviation Reference Models
Calculation of the standard deviation (model variability) of the gcn reference models.

In [ ]:
# get all results
experiment_ids = {'GCN arXiv CS Ref 1':'f48a4e67ea8846a0b3d2ca7e674aca95',
                  'GCN arXiv CS Ref 2':'c4f5985ba24e4c93b569dde82ce15a78',
                  'GCN arXiv CS Ref 3':'942c73672d59449c8a051796ae7f0e49',
                  'GCN arXiv CS Ref 4':'54dba56b65814a4fb588b03d32d61cf6',
                  'GCN arXiv CS Ref 5':'38ea03024c92483aabcf420fa960d293',
                  }
workspace = "swiggy123"
project_name = "link-prediction"

experiment_data = get_experiments_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)
results_data = get_experiments_results_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)

In [ ]:
# best values from above
valid_mrr_values = [0.8484336733818054, 0.8499453663825989, 0.8459517359733582, 0.8498203158378601, 0.8487957119941711]
test_mrr_values = [0.844505250453949, 0.8478335738182068, 0.8422207236289978, 0.8466485142707825, 0.8474210500717163]
valid_mrr_mean = np.round(np.mean(valid_mrr_values),4)
valid_mrr_std = np.round(np.std(valid_mrr_values),4)
test_mrr_mean = np.round(np.mean(test_mrr_values),4)
test_mrr_std = np.round(np.std(test_mrr_values),4)

# get mean valid and test mrr for the 5 runs
print(f"Valid mean MRR: {valid_mrr_mean} ({valid_mrr_std})")
print(f"Test mean MRR: {test_mrr_mean} ({test_mrr_std})")

In [ ]:
# get train val mrr difference over all epochs per run
for i in range(0,5):
    df_fine_val_train_mrr = experiment_data[i]['metrics']
    df_fine_val_train_mrr = df_fine_val_train_mrr[df_fine_val_train_mrr["metricName"].isin(["train_mrr","valid_mrr"])]
    print(np.round(np.mean(np.array(df_fine_val_train_mrr.pivot(columns=["metricName"])['metricValue']['valid_mrr'],dtype=float) - np.array(df_fine_val_train_mrr.pivot(columns=["metricName"])['metricValue']['train_mrr'],dtype=float)),4))


In [ ]:
# plot the 5 runs with an interval
# Combine all runs into a single DataFrame
all_metrics = pd.concat([run['metrics'] for run in experiment_data])

# Convert metricValue to float
all_metrics['metricValue'] = all_metrics['metricValue'].astype(float)

# Group by epoch and metricName to calculate mean and std
agg_metrics = all_metrics.groupby(['epoch', 'metricName']).agg(
    mean_metricValue=('metricValue', 'mean'),
    std_metricValue=('metricValue', 'std')
).reset_index()

plt.figure(figsize=(10, 6))

# Plotting train_mrr with std
train_mrr = agg_metrics[agg_metrics['metricName'] == 'train_mrr']
sns.lineplot(data=train_mrr, x="epoch", y="mean_metricValue", label="train_mrr", color="green")
plt.fill_between(train_mrr['epoch'], 
                 train_mrr['mean_metricValue'] - train_mrr['std_metricValue'],
                 train_mrr['mean_metricValue'] + train_mrr['std_metricValue'],
                 color="green", alpha=0.2)

# Plotting valid_mrr with std
valid_mrr = agg_metrics[agg_metrics['metricName'] == 'valid_mrr']
sns.lineplot(data=valid_mrr, x="epoch", y="mean_metricValue", label="valid_mrr", color="blue")
plt.fill_between(valid_mrr['epoch'], 
                 valid_mrr['mean_metricValue'] - valid_mrr['std_metricValue'],
                 valid_mrr['mean_metricValue'] + valid_mrr['std_metricValue'],
                 color="blue", alpha=0.2)

# Plotting valid_mrr with std
test_mrr = agg_metrics[agg_metrics['metricName'] == 'test_mrr']
sns.lineplot(data=test_mrr, x="epoch", y="mean_metricValue", label="test_mrr",linestyle="dashed", color="red")
plt.fill_between(test_mrr['epoch'], 
                 test_mrr['mean_metricValue'] - test_mrr['std_metricValue'],
                 test_mrr['mean_metricValue'] + test_mrr['std_metricValue'],
                 color="red", alpha=0.2)


plt.xlabel('Epochs')
plt.ylabel('MRR')
plt.title('arXiv CS reference GCN - MRR Curves')
plt.legend()
plt.show()


## Pretrain Models

In [ ]:
experiment_ids = {'GCN Pretrain':'1f18979244fa4176bead16411753dd65',
                  'GIN Pretrain':'98d66fa9995347e1a77ccb90a0116bfd',
                  'SAGE Pretrain':'f2551a3b9725426699c4ea698d264510'}
workspace = "tmandelz-outlook-com's Organisation"
project_name = "transfer-learning-link-prediction"

experiment_data = get_experiments_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)
results_data = get_experiments_results_data(workspace=workspace,project_name=project_name,experiment_ids=experiment_ids)

In [40]:
gcn_df_pretrain = experiment_data[0]['metrics']
firstepoch_test_mrr_gcn_pretrain = float(gcn_df_pretrain[gcn_df_pretrain['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = gcn_df_pretrain.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_gcn_pretrain = float(gcn_df_pretrain.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])


gin_df_pretrain = experiment_data[1]['metrics']
firstepoch_test_mrr_gin_pretrain = float(gin_df_pretrain[gin_df_pretrain['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = gin_df_pretrain.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_gin_pretrain = float(gin_df_pretrain.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])

sage_df_pretrain = experiment_data[2]['metrics']
firstepoch_test_mrr_sage_pretrain = float(sage_df_pretrain[sage_df_pretrain['metricName'] == "test_mrr"].iloc[0]['metricValue'])
valid_max_idx = sage_df_pretrain.query('metricName == "valid_mrr"')["metricValue"].argmax()
finalepoch_test_mrr_sage_pretrain = float(sage_df_pretrain.query('metricName == "test_mrr"')["metricValue"].iloc[valid_max_idx])

## Finetuning (Transfer Learning Metrics)
Calculation the transfer metrics jumpstar, asymptotic performance and transfer ratio.

In [ ]:
# test_mrr_mean is the mean of the best epochs of the reference model
jumpstart_test_gcn = firstepoch_test_mrr_gcn_fine - firstepoch_test_mrr_gcn_ref
asyperf_test_gcn = finalepoch_test_mrr_gcn_fine - finalepoch_test_mrr_gcn_ref
transferratio_test_gcn = (finalepoch_test_mrr_gcn_fine - finalepoch_test_mrr_gcn_ref) / finalepoch_test_mrr_gcn_ref
print(f"GCN: \nJumpstart: {jumpstart_test_gcn}, Asymptotic Performance: {asyperf_test_gcn}, Transfer Ratio: {transferratio_test_gcn}")
print(f"Reference: {finalepoch_test_mrr_gcn_ref}, Pretrain: {finalepoch_test_mrr_gcn_pretrain}, Finetune: {finalepoch_test_mrr_gcn_fine}")

jumpstart_test_gin = firstepoch_test_mrr_gin_fine - firstepoch_test_mrr_gin_ref
asyperf_test_gin = finalepoch_test_mrr_gin_fine - finalepoch_test_mrr_gin_ref
transferratio_test_gin = (finalepoch_test_mrr_gin_fine - finalepoch_test_mrr_gin_ref) / finalepoch_test_mrr_gin_ref
print(f"GIN: \nJumpstart: {jumpstart_test_gin}, Asymptotic Performance: {asyperf_test_gin}, Transfer Ratio: {transferratio_test_gin}")
print(f"Reference: {finalepoch_test_mrr_gin_ref}, Pretrain: {finalepoch_test_mrr_gin_pretrain}, Finetune: {finalepoch_test_mrr_gin_fine}")


jumpstart_test_sage = firstepoch_test_mrr_sage_fine - firstepoch_test_mrr_sage_ref
asyperf_test_sage = finalepoch_test_mrr_sage_fine - finalepoch_test_mrr_sage_ref
transferratio_test_sage = (finalepoch_test_mrr_sage_fine - finalepoch_test_mrr_sage_ref) / finalepoch_test_mrr_sage_ref
print(f"SAGE: \nJumpstart: {jumpstart_test_sage}, Asymptotic Performance: {asyperf_test_sage}, Transfer Ratio: {transferratio_test_sage}")
print(f"Reference: {finalepoch_test_mrr_sage_ref}, Pretrain: {finalepoch_test_mrr_sage_pretrain}, Finetune: {finalepoch_test_mrr_sage_fine}")

